In [23]:
%load_ext watermark
%watermark -a Filippo_Valle -p plotly -m -g -r

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
Author: Filippo_Valle

plotly: 4.14.3

Compiler    : GCC 11.2.0
OS          : Linux
Release     : 5.13.0-21-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 12
Architecture: 64bit

Git hash: 869e8220a6563596d5daace2a5fc2ca488c51402

Git repo: https://github.com/fvalle1/pywalk



In [ ]:
import numpy as np
import plotly.express as px

In [ ]:
from pywalk import run_walk

In [ ]:
walk = run_walk(n_steps = 5000, alpha = 2.)

In [ ]:
px.scatter(walk)

In [ ]:
walk = list(filter(lambda x: x<1e15, walk))

ranks = range(len(walk))

fig = px.scatter(x=range(len(walk)), y=[f/sum(walk) for f in sorted(walk, reverse=True)])
fig.add_scatter(x=list(range(1,1+len(walk))), y=[0.1/_x for _x in range(1,1+len(walk))])

fig.update_layout({"xaxis":{"type":"log"},
                   "yaxis":{"type":"log"}})